In [0]:
import sys
folder = '/content/gdrive/My Drive/Colab Notebooks/my_project1/deep-learning/utils'
#sys.path.append(folder)
sys.path.insert(0, folder)


In [6]:
from google.colab import drive
ROOT = "/content/gdrive"
drive.mount(ROOT, force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
from helper import HelperModel
from dataset_info import DataSet
from dataloader import DataLoader
from model import Net
from transforms import Transform
from train import Train
from test import Test

### Create model and display summary

In [4]:
model_obj = Net()
# create object for Model
device = HelperModel.get_device()

model = model_obj.to(device)

#create object for helper with object
helper_obj = HelperModel(model_obj)

#display summaru of model
helper_obj.display_model_summay(model, (1, 28, 28))



----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 12, 26, 26]             108
       BatchNorm2d-2           [-1, 12, 26, 26]              24
              ReLU-3           [-1, 12, 26, 26]               0
            Conv2d-4           [-1, 12, 24, 24]           1,296
       BatchNorm2d-5           [-1, 12, 24, 24]              24
              ReLU-6           [-1, 12, 24, 24]               0
            Conv2d-7           [-1, 12, 22, 22]           1,296
       BatchNorm2d-8           [-1, 12, 22, 22]              24
              ReLU-9           [-1, 12, 22, 22]               0
           Conv2d-10           [-1, 12, 22, 22]             144
      BatchNorm2d-11           [-1, 12, 22, 22]              24
             ReLU-12           [-1, 12, 22, 22]               0
        MaxPool2d-13           [-1, 12, 11, 11]               0
           Conv2d-14             [-1, 1

/content/gdrive/My Drive/Colab Notebooks/my_project1/deep-learning/utils/model.py:130: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


### Get the mean, std 

In [5]:
data_set_type = "mnist"
#get data set info
data_set_obj = DataSet("mnist", "./data")
mean = data_set_obj.mean
std = data_set_obj.std


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:55: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")


[Train]
 - Numpy Shape: (60000, 28, 28)
 - Tensor Shape: torch.Size([60000, 28, 28])
 - min: tensor(0.)
 - max: tensor(1.)
 - mean: tensor(0.1305)
 - std: tensor(0.3081)
 - var: tensor(0.0949)


### Get the train test transforms 

In [0]:
# Create Image Transform
transform_obj = Transform(mean, std)
train_transforms = transform_obj.train_transforms
test_transforms = transform_obj.test_transforms

### Get Train and Test data set

In [0]:
train_set = data_set_obj.get_train_mnist_dataset(train_transforms)
test_set = data_set_obj.get_test_mnist_dataset(test_transforms)

### Load the train and test data

In [8]:
data_loader_obj = DataLoader(train_set, test_set, batch_size=64)
train_loader = data_loader_obj.train_loader
test_loader = data_loader_obj.test_loader

CUDA Available? True


### Run the model

In [14]:
optimizer = helper_obj.get_l2_regularizer()
EPOCHS = 1
l1_factor=0.0001

train_obj = Train()
test_obj = Test()
train_max_acc = []
train_min_loss = []
for epoch in range(1, EPOCHS+1):
    print("EPOCH:", epoch)
    train_obj.train(model, device, train_loader, optimizer, l1_factor=l1_factor)
    test_obj.test(model, device, test_loader, misclassfied_required=True)
    train_max_acc.append(train_obj.train_acc[-1])
    train_min_loss.append(train_obj.train_losses[-1])
miscassified_images = test_obj.misclassified_images
test_losses = test_obj.test_losses
test_acc = test_obj.test_acc
print("Maximum Training Accuracy = ", max(train_max_acc))
print("Maximum Testing Accuracy = ", max(test_acc))

  0%|          | 0/938 [00:00<?, ?it/s]

EPOCH: 1


/content/gdrive/My Drive/Colab Notebooks/my_project1/deep-learning/utils/model.py:130: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
Train Set: Train Loss=2.306746482849121 Batch_id=937 Accuracy=10.04: 100%|██████████| 938/938 [00:19<00:00, 47.21it/s]



Test set: Average loss: 2.3026, Accuracy: 1010/10000 (10.10%)

Maximum Training Accuracy =  10.04
Maximum Testing Accuracy =  10.1
